# Import Data

In [8]:
import pandas as pd
import numpy as np
import os

In [9]:
DATA_ROOT = '../input/'
ORIGINAL_DATA_FOLDER = os.path.join(DATA_ROOT, 'movie-review-sentiment-analysis-kernels-only')
TMP_DATA_FOLDER = os.path.join(DATA_ROOT, 'kaggle_review_sentiment_tmp_data')

In [10]:
train_data_path = os.path.join(ORIGINAL_DATA_FOLDER, 'train.tsv')
test_data_path = os.path.join(ORIGINAL_DATA_FOLDER, 'test.tsv')
sub_data_path = os.path.join(ORIGINAL_DATA_FOLDER, 'sampleSubmission.csv')

train_df = pd.read_csv(train_data_path, sep="\t")
test_df = pd.read_csv(test_data_path, sep="\t")
sub_df = pd.read_csv(sub_data_path, sep=",")

# EDA

In [11]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [12]:
test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [13]:
sub_df.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


# Data Preprocessing

In [15]:
from keras.preprocessing import text
from keras.preprocessing import sequence
import gensim
from sklearn import preprocessing as skp

Using TensorFlow backend.


In [16]:
max_len = 50
embed_size = 300
max_features = 30000

pretrained_w2v_path = os.path.join(DATA_ROOT, "nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin")

### Tokenize Text

In [17]:
full_text = list(train_df['Phrase'].values) + list(test_df['Phrase'].values)

tk = text.Tokenizer(lower = True, filters='')
tk.fit_on_texts(full_text)
train_tokenized = tk.texts_to_sequences(train_df['Phrase'])
test_tokenized = tk.texts_to_sequences(test_df['Phrase'])


X_train = sequence.pad_sequences(train_tokenized, maxlen = max_len)
X_test = sequence.pad_sequences(test_tokenized, maxlen = max_len)

### Build embedding matrix

In [18]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(pretrained_w2v_path, binary=True).wv

word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = None
    if word in w2v:
        embedding_vector = w2v[word]
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/Users/keyitang/miniconda3/envs/kaggle_review_sentiment/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


### Encode labels

In [19]:
y_train = train_df['Sentiment']

led = skp.LabelEncoder()
led.fit(y_train.values)

y_train = led.transform(y_train.values)

# Define Keras Model

In [20]:
from dlmslib.keras_models import nlp_models
import tensorflow as tf

from keras import callbacks as kc
from keras import optimizers as ko

## CNN Model

In [ ]:
voca_dim = embedding_matrix.shape[0]
time_steps = max_len
output_dim = led.classes_.shape[0]
mlp_dim = 50
num_filters = 5
filter_sizes = [2, 3, 5]
item_embedding = embedding_matrix
mlp_depth = 2

In [ ]:
model = nlp_models.build_cnn_model(
    voca_dim, time_steps, output_dim, mlp_dim, num_filters, filter_sizes, 
    item_embedding=item_embedding, mlp_depth=2 
)

model.summary()

In [ ]:
adam = ko.Adam()
model.compile(adam, loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy",])

file_path = "best_cnn_model.hdf5"
check_point = kc.ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
early_stop = kc.EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
model.fit(X_train, y_train, batch_size=50, epochs=20, validation_split=0.1, callbacks = [check_point, early_stop])

## Attention RNN Model

In [21]:
voca_dim = embedding_matrix.shape[0]
time_steps = max_len
output_dim = led.classes_.shape[0]
rnn_dim = 100
mlp_dim = 100
item_embedding = embedding_matrix
rnn_depth=1
mlp_depth = 2
num_att_channel=1
rnn_drop_out = 0.5
gpu=False

In [22]:
model = nlp_models.build_birnn_attention_model(
    voca_dim, time_steps, output_dim, rnn_dim, mlp_dim, 
    item_embedding=item_embedding, rnn_depth=1, mlp_depth=2, gpu=gpu
)

model.summary()

TypeError: __init__() missing 1 required positional argument: 'axes'

In [ ]:
adam = ko.Adam(clipnorm=2.0)
model.compile(adam, loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy",])

file_path = "best_birnn_attention_model.hdf5"
check_point = kc.ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
early_stop = kc.EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
model.fit(X_train, y_train, batch_size=50, epochs=20, validation_split=0.1, callbacks = [check_point, early_stop])

## RNN-CNN Model